In [31]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors

In [30]:
import fasttext 
src = fasttext.load_model(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.fr.bin")

In [32]:
type(model)
lang = "fr"

In [33]:
dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned

In [ ]:
dst.index_to_key

In [ ]:
src.words

In [34]:
if src.words != dst.index_to_key:
    print("src and dst vocabularies differ. ")
    print("src", len(src.words))
    print("dst", len(dst))
    print("in src, not in dst", set(src.words) - set(dst.index_to_key))
    print("in dst, not in src", set(dst.index_to_key) - set(src.words))

vocab = list(set(src.words) & set(dst.index_to_key))

src and dst vocabularies differ. 
src 1152449
dst 1152449
in src, not in dst {'Ꝃ', 'ᎠᏋ', 'Ꞅ', 'ᏏᏋᎴᎥᏆᎻ', 'ᎠᏆᎴᏓᏰᎢᏋᏒ', 'Ꙍ'}
in dst, not in src {'ꭰꮖꮄꮣᏸꭲꮛꮢ', 'ꭰꮛ', 'ꞅ', 'ꙍ', 'ꝃ', 'ꮟꮛꮄꭵꮖꮋ'}


In [46]:
Y = dst[vocab]

# create X as a matrix of embeddings
X = np.zeros((len(vocab), 300))
for i, word in enumerate(vocab):
    X[i] = src[word]


In [92]:
(test == src.get_input_matrix()).all()

True

In [89]:
test +- 10

array([[-9.799382, -9.624354, -9.755168, ..., -9.410859, -9.015043,
        -9.159589],
       [-9.899751, -9.733056, -9.288855, ..., -9.424584, -9.900037,
        -9.566754],
       [-9.155384, -9.389612, -9.931967, ..., -9.116346, -9.397417,
        -9.909571],
       ...,
       [-9.402906, -9.134227, -9.841292, ..., -9.861008, -9.78985 ,
        -9.930285],
       [-9.479436, -9.702815, -9.910749, ..., -9.717586, -9.280786,
        -9.237035],
       [-9.547465, -9.270367, -9.596948, ..., -9.103327, -9.932954,
        -9.65134 ]], dtype=float32)

In [62]:
test2.shape[0] - test.shape[0]

-2000000

In [85]:
random = np.random.rand(2, 2)
random1 = np.random.rand(2, 2)

res = np.concatenate((random, random1), axis=0)
# change type of random to float32  
res = res.astype(np.float32)

In [88]:
random, random1, res, (res / np.linalg.norm(res, axis=1).reshape(-1,1))

(array([[0.88732813, 0.38475288],
        [0.68760568, 0.89569949]]),
 array([[0.91093964, 0.71704925],
        [0.35371707, 0.59901157]]),
 array([[0.88732815, 0.38475287],
        [0.6876057 , 0.8956995 ],
        [0.91093963, 0.71704924],
        [0.35371706, 0.59901154]], dtype=float32),
 array([[0.91746354, 0.39781985],
        [0.60893494, 0.7932202 ],
        [0.7857687 , 0.6185205 ],
        [0.508469  , 0.86108035]], dtype=float32))

In [91]:
src.set_matrices(input_matrix=test, output_matrix=test2)

In [41]:
type(Y)

numpy.ndarray

In [ ]:
def find_matrix(lang, step=5000):
    
    print("LANG", lang)
    # not aligned
    try:
        src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
        print("Loaded fastText vectors")
    except:
        print("Going to 'vec'")
        src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
    dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned
    
    if src.index_to_key != dst.index_to_key:
        print("src and dst vocabularies differ. ")
        print("src", len(src))
        print("dst", len(dst))
        print("in src, not in dst", set(src.index_to_key) - set(dst.index_to_key))
        print("in dst, not in src", set(dst.index_to_key) - set(src.index_to_key))
    
    vocab = list(set(src.index_to_key) & set(dst.index_to_key))
        
    Y = dst[vocab]
    X = src[vocab]

    W_ = np.linalg.pinv(X) @ Y

    prod = (X @ W_)
    prod = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

    error_couples = []
    right_values = []

    for i in range(0, len(prod), step):
            M = (prod[i:i+step] @ Y.T)
            v = M.argmax(axis=1)

            # sum of the diagonal
            right_values.append(np.diagonal(M[:,v]))
            for j in range(len(v)):
                if v[j] != i+j: # check that the most vector is the word itself
                    print("words do not match", i+j, v[j], M[j,v[j]])
                    print("instead the right word should be ", M[j,j])
                    error_couples.append((i+j, v[j], M[j,v[j]], M[j,j]))

                if M[j,v[j]] < .98:
                    print("small similarity" , i+j, v[j], M[j,v[j]])
    
    return src, dst, X, Y, W_, right_values.mean(), right_values.std(), error_couples

In [ ]:
src, dst, X, Y, W_, mean, std, error_couples = find_matrix("it")

# save the values 
import pickle
with open("it.pkl", "wb") as f:
    pickle.dump((src, dst, X, Y, W_, mean, std, error_couples), f)


In [ ]:
src, dst, X, Y, W_, mean, std, error_couples = find_matrix("it")

with open("it.pkl", "wb") as f:
    pickle.dump((src, dst, X, Y, W_, mean, std, error_couples), f)

In [ ]:
import pickle 

with open("res/it.pkl", "rb") as f:
    X, Y, W_, mean, std, error_couples = pickle.load(f)

In [ ]:
X.shape, Y.shape, W_.shape, mean, std

In [ ]:
error_couples

In [ ]:
lang = "it"

try:
    src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
    print("Loaded fastText vectors")
except:
    print("Going to 'vec'")
    src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned

In [ ]:
print(src.vectors_ngrams)

src.vectors.shape, src.vectors_ngrams.shape

In [ ]:
src.index_to_key[1000], src.buckets_word[1000]

In [ ]:
list = [] 
for i, n in enumerate(src.buckets_word): 
    # if in the list there is at least one index in src.buckets_word[1000], take the index 
    if len(set(n) & set(src.buckets_word[1000])) == 0 and i > 1000:
        list.append(i)
    if len(list) > 10:
        break

In [ ]:
for i in list:
    print(src.index_to_key[i])

In [ ]:
from copy import deepcopy

test = deepcopy(src.vectors_ngrams)
print(test.shape)

# change the vectors of the ngrams
for i in range(10):
    src.vectors_ngrams[i] = np.random.rand(300)

In [ ]:
src.vectors_ngrams = np.random.rand(src.vectors_ngrams.shape[0], 300)

In [ ]:
test[0] == src.vectors_ngrams[0]

# test 19/01

In [ ]:
lang = "hi"

print("LANG", lang)
# not aligned
try:
    src1 = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
    print("Loaded fastText vectors")
except:
    print("Going to 'vec'")
    src1 = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
dst1 = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned

if src1.index_to_key != dst1.index_to_key:
    print("src and dst1 vocabularies differ. ")
    print("src", len(src1))
    print("dst1", len(dst1))
    print("in src, not in dst1", set(src1.index_to_key) - set(dst1.index_to_key))
    print("in dst1, not in src1", set(dst1.index_to_key) - set(src1.index_to_key))

vocab1 = sorted(list(set(src1.index_to_key) & set(dst1.index_to_key)))
    
Y1 = dst1[vocab1]
X1 = src1[vocab1]

In [ ]:
import pickle 

with open(f"res/{lang}.pkl", "rb") as f:
    src, dst, X, Y, W_, mean, std, error_couples, vocab = pickle.load(f)

In [ ]:
emb = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin")

In [ ]:
emb.vectors

In [ ]:


(dst.vectors == dst1.vectors).all()

In [ ]:
vocab1 = list(set(src1.index_to_key) & set(dst1.index_to_key))
vocab = list(set(src.index_to_key) & set(dst.index_to_key))

In [ ]:
vocab1 == vocab

In [ ]:
(src[vocab] == X)

In [ ]:
(X == X1).all()

In [ ]:
test = X[0]

(X == X1[0]).all(axis=1).nonzero(), (X == X1[1]).all(axis=1).nonzero()

In [ ]:
X.shape, X1.shape

In [ ]:
import gensim

gensim.__version__

In [ ]:
X2 = src[vocab]
X3 = src1[vocab]
(X2 == X1).all(), (X3 == X1).all(), (X2 == X3).all(), (X2 == X).all(), (X3 == X).all()

In [ ]:
X[0][2], X1[0][2]

In [ ]:
vocab = list(set(src1.index_to_key) & set(dst1.index_to_key))
vocab1 = list(set(src.index_to_key) & set(dst.index_to_key))

vocab[0], vocab1[0]

In [ ]:
prod = (X @ W_)
prod_norm = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

In [ ]:
v1 = []
v2 = []
for elem in error_couples:
    v1.append(elem[0])
    v2.append(elem[1])

In [ ]:
for i in v1: 
    print(i, np.linalg.norm(prod[i]), np.linalg.norm(prod_norm[i]))

In [ ]:
for i in v2: 
    print(i, np.linalg.norm(prod[i]), np.linalg.norm(prod_norm[i]))

In [ ]:
for i, j in zip(v1, v2): 
    print(dst.index_to_key[i], dst.index_to_key[j])

# Final Work

In [ ]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import load_facebook_vectors
import pickle

def find_matrix(lang, step=1000):
    
    print("LANG", lang)
    dict = {}

    # not aligned
    try:
        src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 
        print("Loaded fastText vectors")
    except:
        print("Going to 'vec'")
        src = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.vec")
    dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned
    
    if src.index_to_key != dst.index_to_key:
        print("src and dst vocabularies differ. ")
        print("src", len(src))
        print("dst", len(dst))
        print("in src, not in dst", set(src.index_to_key) - set(dst.index_to_key))
        print("in dst, not in src", set(dst.index_to_key) - set(src.index_to_key))
    
    dict["missing_elements"] = [set(src.index_to_key) - set(dst.index_to_key), set(dst.index_to_key) - set(src.index_to_key)]

    vocab = sorted(list(set(src.index_to_key) & set(dst.index_to_key)))
        
    Y = dst[vocab]
    X = src[vocab]

    W_ = np.linalg.pinv(X) @ Y

    prod = (X @ W_)
    prod = prod / np.linalg.norm(prod, axis=1).reshape(-1,1)

    dict["MSE"] = np.square(np.subtract(prod, Y)).sum(axis=1).mean()

    error_couples = []
    right_values = np.array([])

    for i in range(0, len(prod), step):
            #einstein summation for matrix multiplication (einsum) 
            M = (prod[i:i+step] @ Y.T)
            v = M.argmax(axis=1)

            # sum of the diagonal
            right_values = np.concatenate((right_values, np.diagonal(M[:,v])))
            for j in range(len(v)):
                if v[j] != i+j: # check that the most vector is the word itself
                    print("words do not match", i+j, v[j], M[j,v[j]])
                    print("instead the right word should be ", M[j,j])
                    error_couples.append((i+j, v[j], M[j,v[j]], M[j,j]))

                #if M[j,v[j]] < .98:
                #    print("small similarity" , i+j, v[j], M[j,v[j]])
            
            print(i, "/", len(prod), "done") if i % 50_000 == 0 else None
    
    dict["accuracy"] = (right_values.mean(), right_values.std()) 
    dict["n_errors"] = len(error_couples)

    return X, Y, W_, right_values, error_couples, dict

In [ ]:
lang = "hi"

In [ ]:
X, Y, W_, right_values, error_couples, dict = find_matrix(lang)

In [ ]:
dst = KeyedVectors.load_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/aligned/wiki.{lang}.align.vec") # aligned
src = load_facebook_vectors(f"/data1/malto/csavelli/aligned_subwords_fasttext/wiki/wiki.{lang}.bin") 

In [ ]:
src.vectors = src.vectors

In [ ]:
np.linalg.norm(dst.vectors[2])

In [ ]:
type(src)

In [ ]:
(prod*Y).sum(axis=1).mean()

In [ ]:
src.vectors = src.vectors @ W_
src.vectors_ngrams = src.vectors_ngrams @ W_

src.vectors = src.vectors / np.linalg.norm(src.vectors, axis=1).reshape(-1,1)
src.vectors_ngrams = src.vectors_ngrams / np.linalg.norm(src.vectors_ngrams, axis=1).reshape(-1,1)

In [ ]:
# norm of the vectors

np.linalg.norm(src.vectors[2]), np.linalg.norm(src.vectors_ngrams[2])

In [ ]:
# save bin file 
#src.save(f"test.bin")
src.save_word2vec_format(f"test2.bin")
#src.save_word2vec_format(f"/data1/malto/csavelli/aligned_subwords_fasttext/res/wiki.{lang}.vec")

In [ ]:
# load bin file
import gensim

#src2 = gensim.models.fasttext.FastTextKeyedVectors.load(f"test/test.bin", mmap='r')

# load vec file
src3 = KeyedVectors.load_word2vec_format(f"test2.bin")

(src3.vectors[0] == src.vectors[0]).all()